# Professional Financial Services Example

The professional services industry is a highly educated workforce with relatively low levels of productivity. This makes the industry a prime candidate for GAI transformation. As GAI automates routine and well defined tasks, the highly skilled workforce can spend more time on higher value thinking. With GAI, Australia could build on its reputation for high calibre knowledge workers, particularly in financial and legal 
sectors.

![Sector deep-dive: Professional and Financial Services](https://raw.githubusercontent.com/rohit-lakhanpal/build-your-own-copilot/main/docs/img/professional-and-financial-services.png)

References: 
- [Australia’s Generative AI opportunity Research paper | TechCouncil of Australia](https://techcouncil.com.au/wp-content/uploads/2023/07/230714-Australias-Gen-AI-Opportunity-Final-report-vF4.pdf).
- [Combat financial crime with AI and advanced technology from Microsoft](https://www.microsoft.com/en-us/industry/blog/financial-services/2023/06/05/combat-financial-crime-with-ai-and-advanced-technology-from-microsoft/)

|  |  |
| -: | :- |
| Industry | Financial Services |
| Usecase  | Financial Services Advising & Exploring. |
| WHO | Financial Services Sector |
| WHAT | (GAI) for pre-emptive alerting in case of delinquencies, liquidity, market etc. and assist stress-testing through interactive simulations. |
| WOW | Assist clients in maintaining their financial well-being, contributing to a more robust and resilient financial ecosystem. |



> <br />**Note:** <br /> <br /> This is an example to get you going and is not a complete solution. It’s important to note that this example is intended to provide guidance and should be adapted to suit your specific needs. By using this example as a starting point, you can save time and effort while still creating a high-quality document. <br /> <br />

## Basic Setup

This section hides away the complexity of working with the Kernel. 

In [1]:
//#r "nuget: dotenv.net, *-*"
#r "nuget: dotenv.net, 3.1.2"
//#r "nuget: Microsoft.SemanticKernel, *-*"
#r "nuget: Microsoft.SemanticKernel, 0.24.230918.1-preview"
//#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.AzureCognitiveSearch, *-*"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.AzureCognitiveSearch, 0.24.230918.1-preview"
//#r "nuget: Microsoft.SemanticKernel.Skills.Core, *-*"
#r "nuget: Microsoft.SemanticKernel.Skills.Core, 0.24.230918.1-preview"
//#r "nuget: Microsoft.SemanticKernel.Skills.Web, *-*"
#r "nuget: Microsoft.SemanticKernel.Skills.Web, 0.24.230918.1-preview"
// Configurations
#r "nuget: Microsoft.Extensions.Configuration, *-*"
#r "nuget: Microsoft.Extensions.Configuration.FileExtensions, *-*"
#r "nuget: Microsoft.Extensions.Configuration.Json, *-*"
#r "nuget: Microsoft.Extensions.Configuration.EnvironmentVariables, *-*" 
#r "nuget: Microsoft.Extensions.Configuration.Binder, *-*"

Installed Packages dotenv.net, 3.1.2 Microsoft.Extensions.Configuration, 8.0.0-rc.1.23419.4 Microsoft.Extensions.Configuration.Binder, 8.0.0-rc.1.23419.4 Microsoft.Extensions.Configuration.EnvironmentVariables, 8.0.0-rc.1.23419.4 Microsoft.Extensions.Configuration.FileExtensions, 8.0.0-rc.1.23419.4 Microsoft.Extensions.Configuration.Json, 8.0.0-rc.1.23419.4 Microsoft.SemanticKernel, 0.24.230918.1-preview Microsoft.SemanticKernel.Connectors.Memory.AzureCognitiveSearch, 0.24.230918.1-preview Microsoft.SemanticKernel.Skills.Core, 0.24.230918.1-preview Microsoft.SemanticKernel.Skills.Web, 0.24.230918.1-preview

In [4]:
#!import ../config/Setup.cs
#!import ../config/Utils.cs
#!import ../config/KernelHelper.cs

var kernel = Setup.LoadKernel();
kernel.Display();

LoggerFactory
Memory
Skills
PromptTemplateEngine
HttpHandlerFactory


## Scenario 1 

| | |
| -: | :- |
| Overview | Recommendations based on as-is knowledge. |
| Techniques | Role prompting |
| Model | GPT 4 Turbo (0613)  |
| Weights | TopP:0.7; Temp:0.9  |


In [5]:
using System.Text.Json;
var context = kernel.CreateNewContext();
context.Variables["stocks"] = System.Text.Json.JsonSerializer.Serialize(new[] {
    new {symbol = "MSFT", allocation = 0.3},
    new {symbol = "ACN", allocation = 0.1},
    new {symbol = "JPM", allocation = 0.3},
    new {symbol = "PEP", allocation = 0.3}
});
context.Variables["age"] = "35";
context.Variables["income"] = "100000";
context.Variables["riskTolerance"] = "agressive";
context.Variables.Display();

key,value
stocks,"[{""symbol"":""MSFT"",""allocation"":0.3},{""symbol"":""ACN"",""allocation"":0.1},{""symbol"":""JPM"",""allocation"":0.3},{""symbol"":""PEP"",""allocation"":0.3}]"
age,35
income,100000
INPUT,
riskTolerance,agressive


In [10]:

ChatHistory chatHistory;

string systemMessage = @"""
Act as a financial advisor and provide advice based on the given portfolio allocation and user information such as a age, income, and risk.
""";

string userMessage = @"""
For someone who is {{$age}} years old, with {{$income}} as household income, and with a risk tolerance of {{$riskTolerance}}, what is a ideal advice on the given portfolio allocation?  

Your advice should be one of the following outputs: ""Buy"", ""Sell"", or ""Hold"" for each of the stocks listed in the stock allocation below.

[STOCK ALLOCATION]
{{$stocks}}
[END STOCK ALLOCATION]

Respond with a well formed table.
""";

(await KernelHelper.ProcessAsync(kernel, systemMessage, userMessage, context)).LastOrDefault().Content.Display();


Based on the provided portfolio allocation and the given user information, here is my advice:

| Stock Symbol | Allocation | Advice  |
|--------------|------------|---------|
| MSFT         | 30%        | Buy     |
| ACN          | 10%        | Buy     |
| JPM          | 30%        | Hold    |
| PEP          | 30%        | Sell    |

Given the aggressive risk tolerance, the individual can afford to increase their exposure to higher growth stocks such as Microsoft (MSFT) and Accenture (ACN). 

JPMorgan Chase (JPM) is a stable stock in the financial sector and already has a significant allocation, therefore the advice is to hold.

PepsiCo (PEP) is a more defensive stock and given the aggressive risk profile, reducing the allocation by selling some of the holdings in this stock could be beneficial to invest in higher growth potential stocks.

**Now, re-run the same scenario but respond with JSON**

In [12]:

ChatHistory chatHistory;

string systemMessage = @"""
ONLY USE JSON PROPERTIES IN THIS LIST:
[JSON PROPERTY LIST]
portfolio
[END LIST]

[CONTENT]
{{$stocks}}
[END CONTENT]

EMIT WELL FORMED JSON ALWAYS.
BE BRIEF AND TO THE POINT.
""";

string userMessage = @"""
Act as a financial advisor and provide advice based on the given portfolio allocation and user information such as a age, income, and risk.  
For someone who is {{$age}} years old, with {{$income}} as household income, and with a risk tolerance of {{$riskTolerance}}, what is a ideal advice on the given portfolio allocation?  
Your advice should be one of the following outputs: ""Buy"", ""Sell"", or ""Hold"" as show in the example below.  
Return well-formed JSON with an ""gptRecommendation"" property containing your recommendation.  
Example: {""portfolio"":[{""name"":""Stocks"",""gptRecommendation"":""""},{""name"":""Bonds"",""gptRecommendation"":""""},{""name"":""Cash"",""gptRecommendation"":""""},{""name"":""Home Equity"",""gptRecommendation"":""""}]}  
""";

Utils.FormatJsonString((await KernelHelper.ProcessAsync(kernel, systemMessage, userMessage, context)).LastOrDefault().Content)

{
  "portfolio": [
    {
      "symbol": "MSFT",
      "allocation": 0.3,
      "gptRecommendation": "Buy"
    },
    {
      "symbol": "ACN",
      "allocation": 0.1,
      "gptRecommendation": "Hold"
    },
    {
      "symbol": "JPM",
      "allocation": 0.3,
      "gptRecommendation": "Buy"
    },
    {
      "symbol": "PEP",
      "allocation": 0.3,
      "gptRecommendation": "Sell"
    }
  ]
}

## Scenario 2


| | |
| -: | :- |
| Overview | Recommendations based on updated knowledge. |
| Techniques | Role prompting, Update Context Window |
| Model | GPT 4 Turbo (0613)  |
| Weights | TopP:0.7; Temp:0.9  |


Get updated US inflation data.

In [13]:
var webQuery = "What is the inflation rate in the US in September 2023?";

var inflationData = await kernel.Func("bing", "search").InvokeAsync(webQuery);
//Utils.Print(inflationData.Result);

var inflationSummary  = await kernel.Func("conversation", "SummarizeConversation").InvokeAsync(inflationData);
Utils.Print(inflationSummary.Result);

// var inflationTopics  = await kernel.Func("conversation", "GetConversationTopics").InvokeAsync(inflationData);
// Console.WriteLine(inflationTopics.Result);

// var inflationActionItems  = await kernel.Func("conversation", "GetConversationActionItems").InvokeAsync(inflationData);
// Utils.Print(inflationActionItems.Result);

context.Variables["inflationSummary"] = inflationSummary.Result;
context.Variables.Display();

The annual inflation rate in the United States increased to 3.7% for the 12
months ending in August 2023, according to data from the U.S. Labor Department.
This is an acceleration from the 3.2% rise in the previous period and is above
market forecasts of 3.6%. The Consumer Price Index for All Urban Consumers
increased by 0.6% in August, the largest monthly increase in over a year.
However, core inflation, which excludes food and energy, decelerated to 4.3%,
the lowest result since September 2021. Despite the rise in inflation, consumer
expectations for inflation fell in early September to the lowest levels in over
two years, indicating increased optimism about the economic outlook. In response
to the economic conditions, the Federal Reserve implemented four significant 75
basis point rate hikes in the second half of the year. The next update on
inflation is scheduled for release on October 12.



key,value
stocks,"[{""symbol"":""MSFT"",""allocation"":0.3},{""symbol"":""ACN"",""allocation"":0.1},{""symbol"":""JPM"",""allocation"":0.3},{""symbol"":""PEP"",""allocation"":0.3}]"
inflationSummary,"The annual inflation rate in the United States increased to 3.7% for the 12 months ending in August 2023, according to data from the U.S. Labor Department. This is an acceleration from the 3.2% rise in the previous period and is above market forecasts of 3.6%. The Consumer Price Index for All Urban Consumers increased by 0.6% in August, the largest monthly increase in over a year. However, core inflation, which excludes food and energy, decelerated to 4.3%, the lowest result since September 2021. Despite the rise in inflation, consumer expectations for inflation fell in early September to the lowest levels in over two years, indicating increased optimism about the economic outlook. In response to the economic conditions, the Federal Reserve implemented four significant 75 basis point rate hikes in the second half of the year. The next update on inflation is scheduled for release on October 12."
age,35
income,100000
INPUT,
riskTolerance,agressive


In [14]:
using System.Text.Json;
ChatHistory chatHistory;

string systemMessage = @"""
ONLY USE JSON PROPERTIES IN THIS LIST:
[JSON PROPERTY LIST]
portfolio
[END LIST]

[CONTENT]
{{$stocks}}
[END CONTENT]

EMIT WELL FORMED JSON ALWAYS.
BE BRIEF AND TO THE POINT.
""";

string userMessage = @"""
Act as a financial advisor and provide advice based on the given portfolio allocation and user information such as a age, income, and risk.  
For someone who is {{$age}} years old, with {{$income}} as household income, and with a risk tolerance of {{$riskTolerance}}, what is a ideal advice on the given portfolio allocation?  
Your advice should be one of the following outputs: ""Buy"", ""Sell"", or ""Hold"" as show in the example below.  
Return well-formed JSON with an ""gptRecommendation"" property containing your recommendation.  
Example: {""portfolio"":[{""name"":""Stocks"",""gptRecommendation"":""""},{""name"":""Bonds"",""gptRecommendation"":""""},{""name"":""Cash"",""gptRecommendation"":""""},{""name"":""Home Equity"",""gptRecommendation"":""""}]}  

[INFLATION SUMMARY]
{{$inflationSummary}}
[END INFLATION SUMMARY]
""";

(await KernelHelper.ProcessAsync(kernel, systemMessage, userMessage, context)).Display();

index value 0 Microsoft.SemanticKernel.AI.ChatCompletion.ChatHistory+ChatMessage Role system Label system Content "
ONLY USE JSON PROPERTIES IN THIS LIST:
[JSON PROPERTY LIST]
portfolio
[END LIST]

[CONTENT]
[{"symbol":"MSFT","allocation":0.3},{"symbol":"ACN","allocation":0.1},{"symbol":"JPM","allocation":0.3},{"symbol":"PEP","allocation":0.3}]
[END CONTENT]

EMIT WELL FORMED JSON ALWAYS.
BE BRIEF AND TO THE POINT.
" 1 Microsoft.SemanticKernel.AI.ChatCompletion.ChatHistory+ChatMessage Role user Label user Content "
Act as a financial advisor and provide advice based on the given portfolio allocation and user information such as a age, income, and risk. 
For someone who is 35 years old, with 100000 as household income, and with a risk tolerance of agressive, what is a ideal advice on the given portfolio allocation? 
Your advice should be one of the following outputs: "Buy", "Sell", or "Hold" as show in the example below. 
Return well-formed JSON with an "gptRecommendation" property containing your recommendation. 
Example: {"portfolio":[{"name":"Stocks","gptRecommendation":""},{"name":"Bonds","gptRecommendation":""},{"name":"Cash","gptRecommendation":""},{"name":"Home Equity","gptRecommendation":""}]} 

[INFLATION SUMMARY]
The annual inflation rate in the United States increased to 3.7% for the 12 months ending in August 2023, according to data from the U.S. Labor Department. This is an acceleration from the 3.2% rise in the previous period and is above market forecasts of 3.6%. The Consumer Price Index for All Urban Consumers increased by 0.6% in August, the largest monthly increase in over a year. However, core inflation, which excludes food and energy, decelerated to 4.3%, the lowest result since September 2021. Despite the rise in inflation, consumer expectations for inflation fell in early September to the lowest levels in over two years, indicating increased optimism about the economic outlook. In response to the economic conditions, the Federal Reserve implemented four significant 75 basis point rate hikes in the second half of the year. The next update on inflation is scheduled for release on October 12.

[END INFLATION SUMMARY]
" 2 Microsoft.SemanticKernel.AI.ChatCompletion.ChatHistory+ChatMessage Role assistant Label assistant Content {"portfolio":[{"symbol":"MSFT","gptRecommendation":"Hold"},{"symbol":"ACN","gptRecommendation":"Buy"},{"symbol":"JPM","gptRecommendation":"Hold"},{"symbol":"PEP","gptRecommendation":"Hold"}]}

## Scenario 3


| | |
| -: | :- |
| Overview | Recommendations based on updated knowledge. |
| Techniques | Role prompting, Update Context Window, RAG |
| Model | GPT 4 Turbo (0613)  |
| Weights | TopP:0.7; Temp:0.9  |

### 3.1. Prepare Lookup Data-Set

In [15]:
await KernelHelper.PrepareDatabaseAsync(kernel, 
        "../../../data/intelligent-investor.txt", "intelligent-investor-book")

Chunks: 46
Chunk 0 of 46 saved to memory collection intelligent-investor-book
Chunk 1 of 46 saved to memory collection intelligent-investor-book
Chunk 2 of 46 saved to memory collection intelligent-investor-book
Chunk 3 of 46 saved to memory collection intelligent-investor-book
Chunk 4 of 46 saved to memory collection intelligent-investor-book
Chunk 5 of 46 saved to memory collection intelligent-investor-book
Chunk 6 of 46 saved to memory collection intelligent-investor-book
Chunk 7 of 46 saved to memory collection intelligent-investor-book
Chunk 8 of 46 saved to memory collection intelligent-investor-book
Chunk 9 of 46 saved to memory collection intelligent-investor-book
Chunk 10 of 46 saved to memory collection intelligent-investor-book
Chunk 11 of 46 saved to memory collection intelligent-investor-book
Chunk 12 of 46 saved to memory collection intelligent-investor-book
Chunk 13 of 46 saved to memory collection intelligent-investor-book
Chunk 14 of 46 saved to memory collection intel

In [16]:
string vectorSearchQ = "Ben Graham's investment philosophy?";

var vectorResults = await KernelHelper.SearchDatabaseAsync(kernel, 
    "intelligent-investor-book", vectorSearchQ, 2);
vectorResults.ForEach(r => vectorSearchQ += $"\n\n{r.Text}");
vectorResults.Display();
var investmentAdvice = (await kernel.Func("conversation", "SummarizeConversation").InvokeAsync(vectorSearchQ)).Result;
Utils.Print(investmentAdvice);

index value 0 Microsoft.SemanticKernel.Memory.MemoryRecordMetadata IsReference False ExternalSourceName Id intelligent-investor-book-26 Description Dataset: intelligent-investor-book Chunk: 26 Text Projection is dangerous because the future is uncertain, and inflation, economic recessions, pandemics and geopolitical upheavals often arrive without warning.
Graham argues that it is a fool's errand for an ordinary investor to attempt making money by timing the market.
Graham advocates for a protection-based approach that does not try to time the market.
Value investors should identify and invest in large, conservatively financed companies whose present value (as estimated by tangible assets) is substantially below their current stock prices. AdditionalMetadata 26 1 Microsoft.SemanticKernel.Memory.MemoryRecordMetadata IsReference False ExternalSourceName Id intelligent-investor-book-1 Description Dataset: intelligent-investor-book Chunk: 1 Text The Intelligent Investor by Benjamin Graham gives you everything you need to equip yourself with the investor's mindset necessary to avoid the panic of market fluctuations that plague the ordinary investor. Don't be ordinary. Be intelligent.
TOP 20 INSIGHTS
There are two kinds of investors. Defensive investors aim to protect their capital from losses, generate decent returns and minimize frequent decisions.
Enterprising investors devote most of their time to manage their portfolios actively. An enterprising investor does not take more risks than a defensive investor but invests more in stock selection. AdditionalMetadata 1

Ben Graham's investment philosophy emphasizes the dangers of projection due to
the uncertainty of the future, which can be affected by factors such as
inflation, economic recessions, pandemics, and geopolitical upheavals. He
discourages ordinary investors from trying to time the market, instead
advocating for a protection-based approach. Graham suggests that value investors
should focus on large, conservatively financed companies whose present value is
substantially below their current stock prices. His book, "The Intelligent
Investor," provides guidance on developing an investor's mindset to avoid panic
during market fluctuations. Graham also differentiates between two types of
investors: defensive investors, who aim to protect their capital, generate
decent returns, and minimize frequent decisions, and enterprising investors, who
actively manage their portfolios and invest more in stock selection without
necessarily taking more risks.



In [17]:
context.Variables["investmentAdvice"] = investmentAdvice;
context.Variables.Display();

key,value
stocks,"[{""symbol"":""MSFT"",""allocation"":0.3},{""symbol"":""ACN"",""allocation"":0.1},{""symbol"":""JPM"",""allocation"":0.3},{""symbol"":""PEP"",""allocation"":0.3}]"
inflationSummary,"The annual inflation rate in the United States increased to 3.7% for the 12 months ending in August 2023, according to data from the U.S. Labor Department. This is an acceleration from the 3.2% rise in the previous period and is above market forecasts of 3.6%. The Consumer Price Index for All Urban Consumers increased by 0.6% in August, the largest monthly increase in over a year. However, core inflation, which excludes food and energy, decelerated to 4.3%, the lowest result since September 2021. Despite the rise in inflation, consumer expectations for inflation fell in early September to the lowest levels in over two years, indicating increased optimism about the economic outlook. In response to the economic conditions, the Federal Reserve implemented four significant 75 basis point rate hikes in the second half of the year. The next update on inflation is scheduled for release on October 12."
age,35
income,100000
INPUT,
investmentAdvice,"Ben Graham's investment philosophy emphasizes the dangers of projection due to the uncertainty of the future, which can be affected by factors such as inflation, economic recessions, pandemics, and geopolitical upheavals. He discourages ordinary investors from trying to time the market, instead advocating for a protection-based approach. Graham suggests that value investors should focus on large, conservatively financed companies whose present value is substantially below their current stock prices. His book, ""The Intelligent Investor,"" provides guidance on developing an investor's mindset to avoid panic during market fluctuations. Graham also differentiates between two types of investors: defensive investors, who aim to protect their capital, generate decent returns, and minimize frequent decisions, and enterprising investors, who actively manage their portfolios and invest more in stock selection without necessarily taking more risks."
riskTolerance,agressive


### 3.2 Explicitly call Chat Completions

In [18]:
string systemMessage = @"""
ONLY USE JSON PROPERTIES IN THIS LIST:
[JSON PROPERTY LIST]
portfolio
[END LIST]

[CONTENT]
{{$stocks}}
[END CONTENT]

EMIT WELL FORMED JSON ALWAYS.
BE BRIEF AND TO THE POINT.
""";

string userMessage = @"""
Act as a financial advisor and provide advice based on the given portfolio allocation and user information such as a age, income, and risk.  
For someone who is {{$age}} years old, with {{$income}} as household income, and with a risk tolerance of {{$riskTolerance}}, what is a ideal advice on the given portfolio allocation?  
Your advice should be one of the following outputs: ""Buy"", ""Sell"", or ""Hold"" as show in the example below.  
Return well-formed JSON with an ""gptRecommendation"" property containing your recommendation.  
Example: {""portfolio"":[{""name"":""Stocks"",""gptRecommendation"":""""},{""name"":""Bonds"",""gptRecommendation"":""""},{""name"":""Cash"",""gptRecommendation"":""""},{""name"":""Home Equity"",""gptRecommendation"":""""}]}  

[INFLATION SUMMARY]
{{$inflationSummary}}
[END INFLATION SUMMARY]

[INVESTMENT ADVICE]
{{$investmentAdvice}}
[END INVESTMENT ADVICE]
""";

(await KernelHelper.ProcessAsync(kernel, systemMessage, userMessage, context)).Display();

index value 0 Microsoft.SemanticKernel.AI.ChatCompletion.ChatHistory+ChatMessage Role system Label system Content "
ONLY USE JSON PROPERTIES IN THIS LIST:
[JSON PROPERTY LIST]
portfolio
[END LIST]

[CONTENT]
[{"symbol":"MSFT","allocation":0.3},{"symbol":"ACN","allocation":0.1},{"symbol":"JPM","allocation":0.3},{"symbol":"PEP","allocation":0.3}]
[END CONTENT]

EMIT WELL FORMED JSON ALWAYS.
BE BRIEF AND TO THE POINT.
" 1 Microsoft.SemanticKernel.AI.ChatCompletion.ChatHistory+ChatMessage Role user Label user Content "
Act as a financial advisor and provide advice based on the given portfolio allocation and user information such as a age, income, and risk. 
For someone who is 35 years old, with 100000 as household income, and with a risk tolerance of agressive, what is a ideal advice on the given portfolio allocation? 
Your advice should be one of the following outputs: "Buy", "Sell", or "Hold" as show in the example below. 
Return well-formed JSON with an "gptRecommendation" property containing your recommendation. 
Example: {"portfolio":[{"name":"Stocks","gptRecommendation":""},{"name":"Bonds","gptRecommendation":""},{"name":"Cash","gptRecommendation":""},{"name":"Home Equity","gptRecommendation":""}]} 

[INFLATION SUMMARY]
The annual inflation rate in the United States increased to 3.7% for the 12 months ending in August 2023, according to data from the U.S. Labor Department. This is an acceleration from the 3.2% rise in the previous period and is above market forecasts of 3.6%. The Consumer Price Index for All Urban Consumers increased by 0.6% in August, the largest monthly increase in over a year. However, core inflation, which excludes food and energy, decelerated to 4.3%, the lowest result since September 2021. Despite the rise in inflation, consumer expectations for inflation fell in early September to the lowest levels in over two years, indicating increased optimism about the economic outlook. In response to the economic conditions, the Federal Reserve implemented four significant 75 basis point rate hikes in the second half of the year. The next update on inflation is scheduled for release on October 12.

[END INFLATION SUMMARY]

[INVESTMENT ADVICE]
Ben Graham's investment philosophy emphasizes the dangers of projection due to the uncertainty of the future, which can be affected by factors such as inflation, economic recessions, pandemics, and geopolitical upheavals. He discourages ordinary investors from trying to time the market, instead advocating for a protection-based approach. Graham suggests that value investors should focus on large, conservatively financed companies whose present value is substantially below their current stock prices. His book, "The Intelligent Investor," provides guidance on developing an investor's mindset to avoid panic during market fluctuations. Graham also differentiates between two types of investors: defensive investors, who aim to protect their capital, generate decent returns, and minimize frequent decisions, and enterprising investors, who actively manage their portfolios and invest more in stock selection without necessarily taking more risks.
[END INVESTMENT ADVICE]
" 2 Microsoft.SemanticKernel.AI.ChatCompletion.ChatHistory+ChatMessage Role assistant Label assistant Content {"portfolio":[{"symbol":"MSFT","gptRecommendation":"Hold"},{"symbol":"ACN","gptRecommendation":"Buy"},{"symbol":"JPM","gptRecommendation":"Hold"},{"symbol":"PEP","gptRecommendation":"Buy"}]}

![Memory Orchestration](https://raw.githubusercontent.com/Azure-Samples/miyagi/main/assets/images/sk-memory-orchestration.png)
